### This notebook can only be run in colab!

In [ ]:
#Check if colab is running
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %tensorflow_version 2.x
    
#import TF  
import tensorflow as tf
from platform import python_version
print("Tensorflow version", tf.__version__)
print("Python version =",python_version())

In [2]:
import h5py
import numpy as np
np.random.seed(1338)

from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams.update({
    "figure.constrained_layout.use" : True,
    "font.size" : 14,
    "figure.figsize" : (7, 5)
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Input

In [6]:
flowers = [
    "ButterflyBush",
    "GrapeHyacinth",
    "BalloonFlower",
    "BleedingHeart",
    "FrangipaniFlower",
    "Daisy",
    "Black-eyedSusan",
    "BlanketFlower",
    "Waterlilies"
]
img_rows = 200
img_cols = 200
shape_ord = (img_rows, img_cols, 3)


with h5py.File("data/images_resized_train.h5", "r") as file:
    X = np.array(file["/images"]).astype("float32")
    y = np.array(file["/labels"]).astype("uint8")
    
X.shape, y.shape

((2441, 200, 200, 3), (2441,))

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y))
y = to_categorical(y, num_classes)

print(y[0])
print("Für die einzelnen Klassen sind so viele Instanzen im Datensatz:\n", 
      flowers, "\n", np.sum(y, axis=0))

In [7]:
# Split 15% of total data as validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1764, random_state=42, stratify=y)
X_train.shape

(2010, 200, 200, 3)

# Plotting and evaluate function

In [8]:
# Plotting functions adapted from exercise 5
import itertools

def plot_history(network_history):
    fig, [ax1, ax2] = plt.subplots(ncols=2, figsize=(15, 5))
    ax1.set(
      xlabel = "Epochs",
      ylabel = "Loss" 
    )
    ax1.plot(network_history.history['loss'], label="Training")
    ax1.plot(network_history.history['val_loss'], label="Validation")
    ax1.legend()
    ax1.grid()

    ax2.set(
      xlabel = "Epochs",
      ylabel = "Accuracy" 
    )
    ax2.plot(network_history.history['accuracy'], label="Training")
    ax2.plot(network_history.history['val_accuracy'], label="Validation")
    ax2.legend()
    ax2.grid()
    fig.show()
    

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


def display_errors(errors_index, img_errors, pred_errors, obs_errors, img_rows, img_cols):
    """ This function shows 6 images with their predicted and real labels """
    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(10, 8))
    for i, ax in enumerate(axs.flat):
      error = errors_index[i]
      ax.imshow(img_errors[error].reshape(img_rows, img_cols, 3),
                interpolation='nearest')
      ax.set_title(f"Predicted label :{pred_errors[error]}\nTrue label :{obs_errors[error]}")
    
    fig.show()


In [9]:
from sklearn.metrics import confusion_matrix, classification_report

def evaluate(X_test, Y_test, label):
    label_name = flowers[label]

    ### Evaluate loss and metrics
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print('Test Loss:', loss)
    print('Test Accuracy:', accuracy)
    # Predict the values from the test dataset
    Y_pred = model.predict(X_test)
    # Convert one hot vectors to classes 
    Y_cls = np.argmax(Y_pred, axis = 1)
    Y_true = np.argmax(Y_test, axis = 1) 
    print('Classification Report:\n', classification_report(Y_true, Y_cls))
    
    ### Plot 0 probability
    Y_pred_prob = Y_pred[:, label]
    plt.figure(figsize=(8, 5))
    plt.hist(Y_pred_prob[Y_true == label], alpha=0.5, color='red', 
             bins=10, log=True, label=f"True label is {label_name}")
    plt.hist(Y_pred_prob[Y_true != label], alpha=0.5, color='blue',
             bins=10, log=True, label=f"True label is not {label_name}")
    plt.legend()
    plt.xlabel(f'Probability of Label {label_name}')
    plt.ylabel('Number of entries')
    plt.show()
    
    ### compute and plot the confusion matrix
    confusion_mtx = confusion_matrix(Y_true, Y_cls) 
    plot_confusion_matrix(confusion_mtx, classes = flowers)

    ### Plot largest errors
    errors = (Y_cls - Y_true != 0)
    Y_cls_errors = Y_cls[errors]
    Y_pred_errors = Y_pred[errors]
    Y_true_errors = Y_true[errors]
    X_test_errors = X_test[errors]

    # Probabilities of the wrong predicted labels
    Y_pred_errors_prob = np.max(Y_pred_errors, axis = 1)
    # Predicted probabilities of the true values in the error set
    true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))
    
    # Difference between the probability of the predicted label and the true label
    delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors
    sorted_dela_errors = np.argsort(delta_pred_true_errors)
    # Top 6 errors 
    most_important_errors = sorted_dela_errors[-6:]
    display_errors(most_important_errors, X_test_errors, Y_cls_errors, Y_true_errors,
                   X_test.shape[1], X_test.shape[2])
    print("Biggest error probabilities are: ", Y_pred_errors_prob[most_important_errors])
    
    ### Plot predictions
    predicted = Y_cls[:15]
    fig, axs = plt.subplots(nrows=3, ncols=5, figsize=(12, 6))

    for i, ax in enumerate(axs.flat):
        ax.imshow(X_test[i].reshape(img_rows, img_cols, 3),
                  interpolation='nearest')
        ax.text(0, 0, flowers[predicted[i]], 
                color='black', bbox=dict(facecolor='white', alpha=1))
        ax.axis('off')

# Train first network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

nb_epoch = 40
batch_size = 128

# number of convolutional filters to use
nb_filters = 64
# number of nodes for dense layers
dense_nodes = 128
# dropout rate
rate_do = 0.3 

# convolution kernel size
nb_conv = 4
#size of pooling area for max pooling
nb_pool = 2


# Manually trying some structures leads to something like this working quite well 
# (2 dense layers does improve the result)
layers_base = [
    Conv2D(nb_filters, kernel_size=nb_conv, padding='valid', activation='relu', input_shape=shape_ord),
    MaxPooling2D(pool_size=nb_pool),
    Conv2D(nb_filters / 2, kernel_size=nb_conv, padding='valid', activation='relu'),
    MaxPooling2D(pool_size=nb_pool),
    Conv2D(nb_filters / 4, kernel_size=nb_conv, padding='valid', activation='relu'),
    MaxPooling2D(pool_size=nb_pool),
    Flatten(),
    Dropout(rate_do),
    Dense(dense_nodes, activation='relu'),
    Dropout(rate_do),
    Dense(dense_nodes, activation='relu'),
    Dropout(rate_do),
    Dense(len(flowers), activation='softmax')
]


model = Sequential(layers_base)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])          
model.summary()

In [ ]:
from tensorflow.keras.callbacks import Callback

class HistoryEpoch(Callback):
    def __init__(self, data):
        self.data = data        
        
    def on_train_begin(self, logs={}):
        self.loss = []
        self.acc = []

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.data
        l, a = self.model.evaluate(x, y, verbose=0)
        self.loss.append(l)
        self.acc.append(a)


train_hist = HistoryEpoch((X_train, y_train))
val_hist = HistoryEpoch((X_val, y_val))

hist = model.fit(X_train, y_train, batch_size = batch_size, 
                 epochs = nb_epoch, verbose=1, 
                 validation_data = (X_val, y_val),
                 callbacks = [val_hist, train_hist])

In [ ]:
plot_history(hist)

# Grid Search

In [ ]:
def build_model_from_params(params):
    layers = [
        Conv2D(params["num_filters"], kernel_size=nb_conv, padding='valid', activation='relu', input_shape=shape_ord),
        MaxPooling2D(pool_size=nb_pool),
        Conv2D(params["num_filters"] / 2, kernel_size=nb_conv, padding='valid', activation='relu'),
        MaxPooling2D(pool_size=nb_pool),
        Conv2D(params["num_filters"] / 4, kernel_size=nb_conv, padding='valid', activation='relu'),
        MaxPooling2D(pool_size=nb_pool),
        Flatten(),
        Dropout(params["dropout"]),
        Dense(params["dense_nodes"], activation='relu'),
        Dropout(params["dropout"]),
        Dense(params["dense_nodes"], activation='relu'),
        Dropout(params["dropout"]),
        Dense(len(flowers), activation='softmax')
    ]
  
    model = Sequential(layers)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 


param_space = {
    'batch_size': [128, 256],
    'num_filters': [32, 64],
    'dropout': [0.3, 0.4, 0.5],
    'dense_nodes': [64, 128]
}

value_combis = itertools.product(*[v for v in param_space.values()])

param_combis = [{key:value for key, value in zip(param_space.keys(), combi)} for combi in value_combis]

print(f"Es werden {len(param_combis)} Parameterkombination getestet:")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

search_results = []

for idx, params in enumerate(param_combis):
    print(f"Starte Run {idx+1}/{len(param_combis)} mit den Parameteren: {params}")

    string_config = ""
    for key, value in params.items():
      string_config += "_" + key + "=" + str(value)

    # save best model according to validation accuracy
    filepath = f"paramsearch{string_config}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor="val_accuracy", verbose=0, save_best_only=True, mode="max"
    )

    model = build_model_from_params(params)
    
    # train the model
    fit_results = model.fit(
        x = X_train, y = y_train, 
        batch_size = params["batch_size"], epochs = nb_epoch, 
        validation_data = (X_val, y_val), 
        callbacks = [checkpoint],
        verbose = 0
    )

    # extract the best validation scores
    best_val_epoch    = np.argmax(fit_results.history['val_accuracy'])
    best_val_acc      = np.max(fit_results.history['val_accuracy'])
    best_val_acc_loss = fit_results.history['val_loss'][best_val_epoch]

    # get correct training accuracy
    best_model = load_model(filepath)
    best_val_acc_train_loss, best_val_acc_train_acc = best_model.evaluate(X_train, y_train)

    # store results
    search_results.append({
        **params,
        'best_val_acc': best_val_acc,
        'best_val_loss': best_val_acc_loss,
        'best_train_acc': best_val_acc_train_acc,
        'best_train_loss': best_val_acc_train_loss,
        'best_val_epoch': best_val_epoch
    })

In [ ]:
resultsDF = pd.DataFrame(search_results)
resultsDF.sort_values('best_val_acc', ascending=False)